In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/raw/tier_1_data.csv', index_col=False)

In [3]:
df = df.drop(columns=["url", "Unnamed: 0", "event", "match-id", "date", "team1-id", "team2-id"])

In [3]:
df['team1'].unique()

array(['Imperial', 'paiN', 'MIBR', '9z', 'Monte', 'Metizport', 'FURIA',
       'MOUZ', 'Astralis', 'Liquid', 'Virtus.pro', 'G2', 'FaZe',
       'FlyQuest', 'Cloud9', 'HEROIC', 'Steel Helmet', 'Lynn Vision',
       'Wildcard', 'TYLOO', 'Spirit', 'ENCE', 'Falcons', 'Natus Vincere',
       'Complexity', 'GamerLegion', 'Eternal Fire', 'Vitality', 'BIG',
       'OG', 'Ninjas in Pyjamas', 'Apeks', 'TheMongolz', 'BESTIA', 'M80',
       'Nigma Galaxy', 'Fluxo Demons', '9 Pandas Fearless',
       'NAVI Javelins', 'HSG fe', 'FURIA fe', 'FlyQuest RED', 'BetBoom',
       'HAVU', 'Nouns', '3DMAX', '9 Pandas', 'PARIVISION',
       'Movistar Riders', 'Wings Up'], dtype=object)

In [4]:
def convert_match_stage(ms_str):
    if 'group' in ms_str.lower():
        return 'G'
    elif 'quarter' in ms_str.lower():
        return 'QF'
    elif 'semi' in ms_str.lower():
        return 'SF'
    elif 'grand' in ms_str.lower():
        return 'GF'
    else:
        return 'NA'

def convert_match_type(mt_str):
    if '1' in mt_str.lower():
        return 'BO1'
    elif '3' in mt_str.lower():
        return 'BO3'
    elif '5' in mt_str.lower():
        return 'BO5'
    else:
        return 'NA'

In [5]:
df['match_stage'] = df['match_stage'].apply(convert_match_stage)
df['match_type'] = df['match_type'].apply(convert_match_type)
for i in range(10):
    df[f'player{i}_kast'] = df[f'player{i}_kast'].apply(lambda l: np.float32(l.strip('%'))/100)

In [6]:
df['team_1_won'] = np.int32(df['team1score']>df['team2score']) ## maybe consider doing ratio showing how much they won by 

In [54]:
from sklearn.preprocessing import OneHotEncoder

# categorical_features = ['team1', 'team2', 'match_type', 'match_stage', 'player0_id', 'player1_id', 'player2_id', 'player3_id', 'player4_id', 'player5_id', 'player6_id', 'player7_id', 'player8_id', 'player9_id']
categorical_features = ['team1']

encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(df[categorical_features])

numerical_features = ['team1_Ranking', 'team2_Ranking']
# for i in range(10):
#     numerical_features.append(f'player{i}_rating')
#     numerical_features.append(f'player{i}_kast')
#     numerical_features.append(f'player{i}_adr')

X = pd.concat([pd.DataFrame(encoded_features), df[numerical_features]], axis=1)
X.columns = X.columns.astype(str)

y = df['team_1_won']


In [56]:
df['team1'].unique()
df['team2'].unique()

array(['FaZe', 'ENCE', 'MOUZ', 'Ninjas in Pyjamas', 'Wings Up',
       'Lynn Vision', 'Astralis', 'TYLOO'], dtype=object)

In [34]:
# X_test = X[156:]
# y_test = y[156:]
# X_train = X[:156]
# y_train = y[:156]

In [50]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)


In [51]:
X_train

,team1_Ranking,team2_Ranking
154,12,29
63,12,29
54,12,2
107,4,27
50,12,29
166,4,12
58,2,1
48,22,29
88,4,12
21,2,118


In [52]:
from sklearn.linear_model import LogisticRegression

# Initialize and train logistic regression model
log_reg_model = LogisticRegression(C = 1)
log_reg_model.fit(X_train, y_train)

# Predictions
y_pred = log_reg_model.predict(X_test)

In [53]:
from sklearn.metrics import f1_score

acc = np.mean(y_pred == y_test)
f1s = f1_score(y_test, y_pred)

print('acc: ', acc, 'f1 score: ', f1s)


acc:  0.8455882352941176 f1 score:  0.8235294117647058


In [39]:
np.mean(np.int32(-0.4*X['team1_Ranking']>0.8*X['team2_Ranking']) == y)

0.5384615384615384